# Analysis of an *in vivo* patch clamp recording experiment
mouse name: 057  
task: NearFarLong  
experimentalist: Berci Andrásfalvy

### 0. Preprocessing the data
The goal of preprocessing is to load the data recorded by the Axon Instruments software, perform the action potential detection and save it in a way that it can be loaded into the behavioral analysis routines. The only problematic part is the spike detection, since the baseline voltage fluctuates a lot and AP amplitudes are not very high, so the amplitudes are comparable to the baseline fluctuations. These fluctuation can be removed by a high pass filter, still the amplitude of the high frequency noise can be similar to the amplitude of the APs. Therefore this step can not be fully automatized. 

In this case I did the preprocessing using custom-written python scripts that seemed to work fine for this particular recording. The plan is that Bazsi will write a program that allows Berci to change the parameters of the voltage filter and the threshold of the spike detection to best match the particular recording session. This will be the same program as the one that saves the behavioral and the e-phys data in igor-compatible format. I also downsampled the original recordings from 50 000 Hz to 5000 Hz - this should be fine for most of the analysis we want to implement here. 

### 1. Load the necessary object class
To analyse spatial tuning of the recorded cells we use the same program as we use for the *in vivo* imaging data. The name of the functions reflect their primary usage for imaging. The main difference here is that 
1. We have higher time resolution voltage recording instead of the fluorescence signal.
2. For spikes, we have the actual spike times and not just spike probabilities or inferred spike times
3. When we calculate spatial tuning we use the original, 5000 Hz temporal resolution. However, when I calculate shuffling I downsample all data to 50 Hz, and compare the actual data with the shuffled at this, lower resolution. This is mainly due to a memory issue: Shuffling the spike data recorded at 5000 Hz for 300 s (1e6 datapoint) with 1000 shuffles is 1e9 datapoints is a few GB array even for a single neuron. Storing such large array in memory could slow down the calculation. 

We use a custom-made class, ImagingSessionData, that will contain all behavioral and imaging data

In [16]:
from ImageAnal import *
%matplotlib widget

### 2. Tell python where the data is
The required file structure is the following:  
- All data should be in the data folder
- Within the data folder separate subfolders are needed for each mouse. Folder name starts with the **name** of the mouse.
- For each mouse there should be a single folders containing **both** the **e-phys** data and the **behavioral** data.
- The folder is named as `MouseName_TaskName` - so we need a separate folder for each different task
- The behavioral log files are in separate subfolders named by the experiment's start time within the behavioral folder - e.g. `2021-02-03_10-15-50`
- There are several recordings (traces) within the same session. They should be numbered - `imaging_logfile_name` is the number we want to process.
- We need to tell python that the e-phys files are in the same folder. Remember, python thinks that these ar eimaging files!

In [17]:
datapath = os.getcwd() + '/' #current working directory - look for data and strings here!
date_time = '2021-06-25_13-04-27' # date and time of the imaging session
name = 'AB057' # mouse name
task = 'NearFarLong' # task name

suite2p_folder = datapath + 'data/' + name + '_NearFarLong/2021-06-25_13-04-27/' # the e-phys files should be here

## the name and location of the trigger voltage file
TRIGGER_VOLTAGE_FILENAME = suite2p_folder + 'Trigger_0001.csv'

## the name and location of the imaging log file - this is the number of the recording session...
imaging_logfile_name = '0001' # elfiz_session_number


### 3. Load all the data - this taks ~20 secs in my computer
Python looks for the data in the specified folders. It loads the behavioral data (position, lick and rewards) as well as the imaging data. It calculates the activity of the cells as a function of position in the different corridors and calculates their spatial tuning measures and corridor selectivity.  
The name of the object that contains all the data is `D1` here - Data 1.  

**Important**: to tell python that you want to load e-phys data, you need to write at the end that `elfiz = True`  

**Important 2.**: Berci recorded from the same mouse for ~2 hours, and there are almost 1000 laps. Loading all the data needs a lots of memory, and takes forever. So I selected only the 20-80 laps that contain the laps during the current recordings. Iy you want to analyse a different recording, then you need to load the appropriate traces!


In [3]:
# 3. load all the data - this taks ~20 secs in my computer
D1 = ImagingSessionData(datapath, date_time, name, task, suite2p_folder, imaging_logfile_name, TRIGGER_VOLTAGE_FILENAME, selected_laps=np.arange(20,80), speed_threshold=5, randseed=123, elfiz=True)#, startendlap=[27, 99])

trigger logfile loaded
trigger voltage signal loaded
triggers after: 22
n_extra_indexes 5
candidate log indexes [48, 147, 246, 344, 345, 443, 444, 542, 543, 641, 642, 740, 839, 938]
min recorded trigger length: 0.4898000000000007
relevant behavior located, lap time of the first frame: 1360.813762 , log reference index: 48
elfiz data loaded
elfiz time axis loaded
ExpStateMachineLog time interval > 1s:  293  times
############################################################
substage change detected!
first lap in substage  ['1'] is lap 0 , which started at t 1051.0210371398189
the time of the change in imaging time is:  -309.7927248601811
############################################################
calculating rate, reliability and Fano factor...
calculating Skaggs spatial info...
calculating proportion of active laps...
calculating proportion of active laps based on dF/F ...
calculating linear tuning specificity ...
calculating rate, reliability and Fano factor...
calculating Skaggs spat

/Users/ubi/Projects/KOKI/VR/MiceData/utils.py:17: RuntimeWarning: invalid value encountered in true_divide
  r = np.divide(r_num, r_den, out=out_vec, where=vec_nonzero)


The behavior is divided into laps (trials or runs). You can check the **number of laps** and which lap is associated with imaging data in the following way:

In [4]:
print(D1.n_laps)
print(D1.i_Laps_ImData)

60
[28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50]


So we have 229 laps and laps 83-138 contain imaging data.  
### 4. Plotting the behavioral data
You can plot the behavioral data of the session. This mouse runs a lot, but the performance is not great: it lick almost everywhere and does not systematically slow down before reward zone.

In [5]:
#D1.plot_session()
D1.plot_session(selected_laps=D1.i_Laps_ImData)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 6.  Plot the activity lap by lap
We can also plot the lap by lap activity of a the recorded cell. Again, there are several options, but first is to plot the Vm and the spikes as a function of time. To achieve this, we need to set `signal  = 'dF' `.  

Different colors are different laps. Solid line is the recorded Vm - gaps correspond to calibration pulses that have been removed during preprocessing. Traces are shifted so that their minimum is at the corresponding point in the y-axis. Vm curves are transparentm so they do not cover each other. Open square is the reward time and colored dotes are the spikes - each at the y coordinate at the given lap. 

In [20]:
D1.plot_cell_laps(cellid=0, signal='dF') ## look at lap 20

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We see that in lap 6 in corridor 15 the cell fired a lots of spikes. We can check the index of this lap using the following function:

In [22]:
D1.get_lap_indexes(corridor=15, i_lap=0) 

lap # in corridor 15 with imaging data;    lap # within session
0 	 28


We can also plot the activity in this particular lap. The argument `th = -0.5` tells python to include all cells with spike count larger than `-0.5` which is all neurons. `flue = True` means that the flourescence (voltage) is also plotted.

In [26]:
D1.ImLaps[47].plot_tx(th=-0.5, fluo=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
/Users/ubi/Projects/KOKI/VR/MiceData/ImageAnal.py:2119: RuntimeWarning: divide by zero encountered in true_divide
  
/Users/ubi/Projects/KOKI/VR/MiceData/ImageAnal.py:2119: RuntimeWarning: invalid value encountered in true_divide
  


In this lap, the animal was running during the spikes, but this not true for all laps. For examplde, in lap 1 in corridor 15 the animal was still during most of the big burst.

In [19]:
D1.get_lap_indexes(corridor=15, i_lap=1) 
D1.ImLaps[30].plot_tx(th=-0.5, fluo=True)

lap # in corridor 15 with imaging data;    lap # within session
1 	 30


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2-D array with a single row if you intend to specify the same RGB or RGBA value for all points.


We can also see the firing rate a s afunction of the position. Note, that here we only include timesteps where the speed was larger than 5 cm/s. It is possible to change the speed threshold at the beginning of the analysis when defining the ImagingSessionData.

In [15]:
D1.plot_cell_laps(cellid=0, signal='rate') 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 7. Calculate significance of tuning by shuffling the imaging data 
Finally, we can calculate the significance of the tuning of the cell by shuffling its activity 1000 times. This takes a while, but since we only do this for 1 cell it is not too long. But it is only calculated once, and then it automatically reads it from a file. The cell is not significantly spatially tuned, not corridor selective and does not show similar spatial activity in the two corridors.

In [13]:
cellids = np.array([0])
D1.calc_shuffle(cellids, 1000, 'shift')

loading shuffling P-values from file...
0
1
selective cells: []
similar cells: []


In [14]:
print(D1.tuned_cells)

[array([], dtype=int64), array([], dtype=int64)]


In [18]:
D1.save_data()

Session parameters written into file:  cell_properties_corridor_14_N1.csv
cell properties for corridor 14  saved into file:  /Users/ubi/Projects/KOKI/VR/MiceData/data/AB057_NearFarLong/2021-06-25_13-04-27/analysed_data/cell_properties_corridor_14_N1.csv
Session parameters written into file:  cell_properties_corridor_15_N1.csv
cell properties for corridor 15  saved into file:  /Users/ubi/Projects/KOKI/VR/MiceData/data/AB057_NearFarLong/2021-06-25_13-04-27/analysed_data/cell_properties_corridor_15_N1.csv
Session parameters written into file:  ratemaps_corridor_14_N1.csv
ratemap for corridor 14  saved into file:  /Users/ubi/Projects/KOKI/VR/MiceData/data/AB057_NearFarLong/2021-06-25_13-04-27/analysed_data/ratemaps_corridor_14_N1.csv
Session parameters written into file:  ratemaps_corridor_15_N1.csv
ratemap for corridor 15  saved into file:  /Users/ubi/Projects/KOKI/VR/MiceData/data/AB057_NearFarLong/2021-06-25_13-04-27/analysed_data/ratemaps_corridor_15_N1.csv
Session parameters written i